In [1]:
import sys
import os
sys.path.append('../src') # Needed for right import from FairModels

from datetime import datetime
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split
from fairlearn.reductions import ExponentiatedGradient, DemographicParity
from fairlearn.metrics import demographic_parity_difference
from sklearn.tree import DecisionTreeClassifier
import argparse
import random
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import optuna

from load_data import DATALOADER

# Import models
from src.FairModels.ICVAE import ICVAE
from src.FairModels.BinaryMI import BinaryMI
from src.FairModels.VFAE import VFAE

2025-10-28 11:34:57.461565: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/philippwolf/combined_project/venv/lib/python3.9/site-packages/torch/_functorch/deprecated.py:61: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.vmap is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.vmap instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('vmap', 'torch.vmap')


In [2]:
MODELS = {
    "BinaryMI": BinaryMI,
    "VFAE": VFAE,
    "ICVAE": ICVAE
}


HYPERPARAMS = {
    "BinaryMI": [('num_hidden_layers',"int",(2,5)),('size_hidden_layers',"int",(5,30)),("kernel_regularizer","categorical",[0,0.01,0.001]),
                 ("drop_out","categorical",[0.0,0.2,0.5]),("set_quantized_position","fixed",True),("run_eagerly","fixed",False),
                 ("batch_size","fixed",256),("epoch","int",(10,50))],
    "VFAE": [('num_hidden_layers',"int",(2,5)),("size_hidden_layers","int",(5,30)),("kernel_regularizer","categorical",[0,0.01,0.001]),
                 ("drop_out","categorical",[0.0,0.2,0.5]),("batch_size","fixed",256),("epoch","int",(10,50)),("dim_z","int",(5,30))],
    "ICVAE": [('num_hidden_layers',"int",(2,5)),('size_hidden_layers',"int",(5,30)),("kernel_regularizer","categorical",[0,0.01,0.001]),
                ("drop_out","categorical",[0.0,0.2,0.5]),("batch_size","fixed",256),("epoch","int",(10,50)),("dim_z","int",(5,30))]
}

In [3]:
def _hp_optimization(model, hps, X, y, S,n_trials, n_folds, random_seed, stratified=True):
    X = X.to_numpy()
    fixed_params = {}
    for hp in hps:
        if hp[1] == "fixed":
            fixed_params[hp[0]] = hp[2]
    
    def objective(trial):
        hp_trial_dict = {}
        
        for hp in hps:
            if hp[1] == "categorical":
                hp_trial_dict[hp[0]] = trial.suggest_categorical(hp[0], hp[2])
            elif hp[1] == "int":
                hp_trial_dict[hp[0]] = trial.suggest_int(hp[0], hp[2][0], hp[2][1])
            elif hp[1] == "float":
                hp_trial_dict[hp[0]] = trial.suggest_float(hp[0], hp[2][0], hp[2][1])
        
        cur_model = model(**hp_trial_dict,**fixed_params)
        
        if stratified:
            kf = StratifiedKFold(n_splits=n_folds, random_state=random_seed, shuffle=True)
        else:
            kf = KFold(n_splits=n_folds, random_state=random_seed, shuffle=True)
        auc = []
        for i, (train_index, test_index) in enumerate(kf.split(X, y)):
            cur_model.fit(X[train_index], y[train_index], S[train_index])
            y_pred = cur_model.predict_proba(X[test_index])
            y_pred = [1 if x > 0.5 else 0 for x in y_pred]
            cur_acc = roc_auc_score(y[test_index], y_pred)
            auc.append(cur_acc)
        return np.mean(auc)
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials)
    return study.best_value ,{**study.best_params, **fixed_params}

In [4]:
def test_loop(model,X_train,X_test,y_train,y_test,S_train,S_test,gammas,path,hps={}, do_hp_opt=True):
    accuracies = pd.DataFrame(columns=["gamma","acc","dp","acc_threshold","dp_threshold","optimal_threshold"])
    accuracies.set_index("gamma", inplace=True)
    hps.append(("",""))

    for gamma in gammas: 
        hps = hps[:-1] # Delete previous gamma value
        hps.append(("gamma","fixed",gamma)) # Set new gamma
        print(f"Gamma: {gamma}")
        
        _, best_hps = _hp_optimization(model, hps, X_train, y_train, S_train, 50, 3, SEED) # HP-Opt. with 50 combinations and 3-fold cv
        cur_model = model(**best_hps)
        cur_model.fit(X_train, y_train, S_train)

        # Get optimal threshold
        X_val, X_test_holdout, y_val, y_test_holdout, s_val, s_test_holdout = train_test_split(X_test, y_test, S_test, test_size=0.66, random_state=SEED)
        t = np.zeros(5)
        for i in range(5):
            y_prob = cur_model.predict_proba(X_val)
            fpr, tpr, thresholds = roc_curve(y_val, y_prob)
            youden_j = tpr - fpr
            optimal_idx = np.argmax(youden_j) 
            t[i] = thresholds[optimal_idx]
        t = t[~np.isinf(t)]
        optimal_threshold = t.mean()

        # Evaluate model and save results
        y_prob = cur_model.predict_proba(X_test)
        y_pred = [1 if x > 0.5 else 0 for x in y_prob]
        acc = accuracy_score(y_test, y_pred)
        dp = demographic_parity_difference(y_test, y_pred, sensitive_features=S_test)

        y_pred = [1 if x > optimal_threshold else 0 for x in y_prob]
        acc_threshold = accuracy_score(y_test, y_pred)
        dp_threshold = demographic_parity_difference(y_test, y_pred, sensitive_features=S_test)

        accuracies.loc[gamma] = acc, dp, acc_threshold, dp_threshold ,optimal_threshold
        accuracies.to_csv(f"{path}/accuracies.csv")

In [5]:
SEED = 123
random.seed(SEED)
tf.random.set_seed(SEED)
np.random.seed(SEED)

dataset = "adult"
model_name = "BinaryMI"
path = f"{dataset}_{model_name}_{datetime.now().strftime('%d-%m-%Y_%H-%M-%S')}"
os.makedirs(path, exist_ok=True)

X_train, X_test, y_train, y_test, S_train, S_test = DATALOADER[dataset](SEED)

model = MODELS[model_name]
hps = HYPERPARAMS[model_name]

gammas = np.linspace(0,1,11)
test_loop(model,X_train,X_test,y_train,y_test,S_train,S_test,gammas,path,hps)

[I 2025-10-28 11:35:21,184] A new study created in memory with name: no-name-6f35d394-1340-4b26-badd-568805919bc5


Gamma: 0.0


[I 2025-10-28 11:36:01,705] Trial 0 finished with value: 0.5001327108343953 and parameters: {'num_hidden_layers': 5, 'size_hidden_layers': 17, 'kernel_regularizer': 0.001, 'drop_out': 0.2, 'epoch': 43}. Best is trial 0 with value: 0.5001327108343953.
[I 2025-10-28 11:36:30,134] Trial 1 finished with value: 0.5000303692905733 and parameters: {'num_hidden_layers': 3, 'size_hidden_layers': 28, 'kernel_regularizer': 0.01, 'drop_out': 0.2, 'epoch': 34}. Best is trial 0 with value: 0.5001327108343953.
[I 2025-10-28 11:36:58,821] Trial 2 finished with value: 0.5 and parameters: {'num_hidden_layers': 3, 'size_hidden_layers': 30, 'kernel_regularizer': 0.01, 'drop_out': 0.0, 'epoch': 44}. Best is trial 0 with value: 0.5001327108343953.
[I 2025-10-28 11:37:40,698] Trial 3 finished with value: 0.5 and parameters: {'num_hidden_layers': 5, 'size_hidden_layers': 27, 'kernel_regularizer': 0, 'drop_out': 0.5, 'epoch': 47}. Best is trial 0 with value: 0.5001327108343953.
[I 2025-10-28 11:38:18,126] Tria

Gamma: 0.1


[I 2025-10-28 11:57:29,212] Trial 0 finished with value: 0.49998221543771626 and parameters: {'num_hidden_layers': 2, 'size_hidden_layers': 9, 'kernel_regularizer': 0.01, 'drop_out': 0.5, 'epoch': 25}. Best is trial 0 with value: 0.49998221543771626.
[I 2025-10-28 11:57:52,274] Trial 1 finished with value: 0.5353148497538581 and parameters: {'num_hidden_layers': 5, 'size_hidden_layers': 12, 'kernel_regularizer': 0, 'drop_out': 0.2, 'epoch': 17}. Best is trial 1 with value: 0.5353148497538581.
[I 2025-10-28 11:58:22,188] Trial 2 finished with value: 0.571083847589621 and parameters: {'num_hidden_layers': 2, 'size_hidden_layers': 14, 'kernel_regularizer': 0.001, 'drop_out': 0.5, 'epoch': 45}. Best is trial 2 with value: 0.571083847589621.
[I 2025-10-28 11:58:42,622] Trial 3 finished with value: 0.5566516321284863 and parameters: {'num_hidden_layers': 2, 'size_hidden_layers': 24, 'kernel_regularizer': 0.001, 'drop_out': 0.5, 'epoch': 25}. Best is trial 2 with value: 0.571083847589621.
[I 

KeyboardInterrupt: 